In [1]:
import gym
import gym_tictactoe

In [2]:
env = gym.make('tictactoe-v0')

In [3]:
env.action_space

Discrete(9)

In [4]:
env.observation_space

Discrete(19683)

In [5]:
initial_state = env.reset()

In [6]:
env.render()

[' ' ' ' ' ']
[' ' ' ' ' ']
[' ' ' ' ' ']


In [7]:
random_move = env.action_space.sample()
observation, reward, done, _ = env.step(random_move)
random_move

5

In [8]:
env.render()

[' ' ' ' ' ']
[' ' ' ' 'X']
[' ' ' ' ' ']


In [9]:
random_move = env.action_space.sample()
observation, reward, done, _ = env.step(random_move)
random_move

0

In [10]:
env.render()

['O' ' ' ' ']
[' ' ' ' 'X']
[' ' ' ' ' ']
